Capstone Deliverable: Data Curation

This notebook documents the process of acquiring, cleaning, merging, and preparing the datasets for the "Assessing Water Risk Exposure of Global Power Plant Infrastructure" project.

The process involves these major steps:

Setup: Cloning the project repository and unzipping geospatial data.

Loading Raw Data: Reading the power plant database, Aqueduct baseline and future risk data, and the geospatial basin data.

Data Merging: Performing a geospatial join to link power plants to water basins, followed by attribute joins to add the detailed risk data.

Data Cleaning: Handling identified data quality issues like special numeric codes, missing values, and anomalies.

Saving the Final Dataset: Exporting the final, clean, and analysis-ready dataset to a Pickle file to streamline all future analysis.

Datasets Used:

WRI Global Power Plant Database (GPPD)
WRI Aqueduct 4.0 Baseline Monthly & Future Annual Data
WRI Aqueduct 4.0 Geospatial Data (File Geodatabase)

In [ ]:
# ===================================================================
# CELL 2: SETUP AND ENVIRONMENT
#
# This cell imports all necessary libraries and prepares the
# Colab environment by cloning the GitHub repository and unzipping
# the necessary geospatial data.
# ===================================================================

# Import necessary libraries
import os
import pandas as pd
import geopandas as gpd
import fiona
import numpy as np

# --- Environment Setup ---

# Clone the GitHub repository if it doesn't already exist in the session
repo_path = 'global-power-plant-database'
if not os.path.exists(repo_path):
    print("Cloning GitHub repository...")
    # This command downloads your project repository into the Colab environment
    !git clone https://github.com/theouterlimitz/global-power-plant-database.git
else:
    print("GitHub repository already exists.")

# Unzip the Geodatabase file if it hasn't been unzipped yet
# IMPORTANT: Make sure you have uploaded your zipped GDB file (e.g., 'GDB.zip') first!
gdb_folder_path = 'GDB'
if not os.path.exists(gdb_folder_path):
    print("\nUnzipping geospatial data...")
    # This command unzips the geodatabase folder.
    # Replace 'GDB.zip' with the actual name of your zip file if different.
    !unzip GDB.zip
else:
    print("\nGeospatial data folder already exists.")

Data Loading

This step loads all raw data sources from their respective files into pandas or geopandas DataFrames. This includes the power plant database, the two tabular Aqueduct risk datasets, and the geospatial basin data from the File Geodatabase.


In [ ]:
# ===================================================================
# CELL 4: LOAD ALL RAW DATA SOURCES
#
# This cell reads all necessary files into memory.
# It assumes you have uploaded the two Aqueduct CSVs and the GDB.zip
# file to your Colab session.
# ===================================================================
print("\n--- Loading All Raw Datasets ---")

try:
    # Load Power Plant Data from the cloned GitHub repository
    gppd_file_path = 'global-power-plant-database/output_database/global_power_plant_database.csv'
    df_power_plants = pd.read_csv(gppd_file_path, low_memory=False)
    print("Successfully loaded: Global Power Plant Database")

    # Load Aqueduct Baseline Monthly Data from the uploaded CSV
    aq_baseline_monthly_path = 'Aqueduct40_baseline_monthly_y2023m07d05.csv'
    df_aq_baseline_monthly = pd.read_csv(aq_baseline_monthly_path)
    print("Successfully loaded: Aqueduct Baseline Monthly Data")

    # Load Aqueduct Future Annual Data from the uploaded CSV
    aq_future_annual_path = 'Aqueduct40_future_annual_y2023m07d05.csv'
    df_aq_future_annual = pd.read_csv(aq_future_annual_path)
    print("Successfully loaded: Aqueduct Future Annual Data")

    # Load the geospatial basin layer from the unzipped File Geodatabase
    gdb_path = 'GDB/Aq40_Y2023D07M05.gdb'
    target_layer_name = 'baseline_annual'
    gdf_basins = gpd.read_file(gdb_path, layer=target_layer_name)
    print("Successfully loaded: Geospatial basin data")

except FileNotFoundError as e:
    print(f"\nERROR: A file was not found. {e}")
    print("Please ensure you have uploaded the two Aqueduct CSVs and your GDB.zip file to this Colab session.")
except Exception as e:
    print(f"An error occurred during data loading: {e}")

Data Merging

This section contains the core data integration logic. First, the detailed basin polygons are simplified. Second, a geospatial join is performed to assign a basin ID (pfaf_id) to each power plant based on its location. Finally, standard attribute joins are used to merge the detailed baseline and future water risk data onto the power plants using the common pfaf_id.



In [ ]:
# ===================================================================
# CELL 6: DATA MERGING (GEOSPATIAL AND ATTRIBUTE JOINS)
# ===================================================================
print("\n--- Merging All Datasets ---")

# --- Step 6.1: Simplify Basin Geometries ---
# The original basin file is split by province/aquifer. We dissolve it
# to get one single, complete shape for each unique water basin (pfaf_id).
print("Simplifying basin geometries...")
basins_simplified = gdf_basins[['pfaf_id', 'geometry']].dissolve(by='pfaf_id', aggfunc='first').reset_index()

# --- Step 6.2: Geospatial Join ---
# Convert the power plant DataFrame to a GeoDataFrame by creating points from lat/lon
print("Converting power plants DataFrame to a GeoDataFrame...")
gdf_power_plants = gpd.GeoDataFrame(
    df_power_plants,
    geometry=gpd.points_from_xy(df_power_plants.longitude, df_power_plants.latitude),
    crs="EPSG:4326"  # Set CRS to WGS84 (standard lat/lon)
)

# Perform the spatial join to find which basin each power plant point is 'within'
print("Performing spatial join...")
gdf_plants_with_basins = gpd.sjoin(gdf_power_plants, basins_simplified, how="left", predicate="within")
print("Spatial join complete.")

# --- Step 6.3: Attribute Joins ---
# Now, merge the tabular water risk data using the common 'pfaf_id' column
print("Merging baseline and future risk data...")
df_merged = pd.merge(gdf_plants_with_basins, df_aq_baseline_monthly, on='pfaf_id', how='left')
df_final = pd.merge(df_merged, df_aq_future_annual, on='pfaf_id', how='left')
print("Attribute joins complete.")

Final Data Cleaning

This section addresses the data quality issues identified during our initial review. This includes handling special numeric codes used for "No Data" and clipping anomalous negative values to ensure the data is clean and ready for analysis.

In [ ]:
# ===================================================================
# CELL 8: FINAL DATA CLEANING
# ===================================================================
print("\n--- Performing Final Data Cleaning ---")

# Replace all instances of -9999 (the "No Data" code) with NumPy's Not a Number (np.nan)
# This ensures missing data is handled correctly by pandas and plotting libraries.
print("Replacing -9999 values with np.nan...")
df_final.replace(-9999, np.nan, inplace=True)

# Address the negative reported generation values. We cap them at 0 as negative generation is not physically meaningful here.
gen_gwh_cols = [col for col in df_final.columns if 'generation_gwh_' in col and 'estimated' not in col]
print("Capping negative reported generation values at 0...")
for col in gen_gwh_cols:
    df_final[col] = df_final[col].clip(lower=0)

# Drop the unnecessary 'index_right' column that was created during the spatial join
if 'index_right' in df_final.columns:
    df_final.drop(columns=['index_right'], inplace=True)
print("Dropped unnecessary columns.")

print("\nFinal cleaning complete!")

Save Final Analytical Dataset

The Data Curation process is complete. The final, merged, and cleaned GeoDataFrame is saved to a Pickle (.pkl) file. This format is efficient and preserves all data types (including the geometry column). By saving this file, all future analysis can begin by simply loading this single file, avoiding the need to re-run this entire pipeline.

In [ ]:
# ===================================================================
# CELL 10: SAVE AND INSPECT THE FINAL DATASET
# ===================================================================
print("\n--- Saving Final Analytical Dataset ---")

# Define the output filename
output_filename = 'analytical_data.pkl'

# Save the final DataFrame to a Pickle file
df_final.to_pickle(output_filename)

print(f"Successfully saved the final, cleaned dataset to '{output_filename}'")
print("\nData Curation phase is complete!")

# --- Final Inspection ---
print("\n--- Final Dataset Overview ---")
print("\nInfo for the final, merged DataFrame:")
df_final.info(verbose=False)  # Using verbose=False for a concise summary

print("\nShape of the final DataFrame (rows, columns):")
print(df_final.shape)

print("\nSample of key columns from the final dataset:")
# Display a small subset of columns to verify the final structure
key_cols_to_display = ['name', 'country', 'primary_fuel', 'capacity_mw', 'pfaf_id', 'bws_01_label', 'bau80_ws_s']
print(df_final[key_cols_to_display].head())